Notes on ode solver

```python
solution = odeint(immune_and_pd_model, y0, t_eval, args=(params, van_func, lzd_func, immune_model))
```

This is where you **actually solve the system of differential equations** (ODEs) for your bacterial and immune dynamics. Here’s what each part does:

---

### 1️⃣ `odeint`

* `odeint` comes from **SciPy** (`scipy.integrate.odeint`).
* It **numerically integrates a system of ODEs** over time.
* You give it:

  1. A **function** that defines the derivatives (`dy/dt`) of all your state variables.
  2. Initial conditions (`y0`) for each variable.
  3. A **time vector** over which to integrate (`t_eval`).
  4. Any extra arguments that your derivative function needs (`args`).

---

### 2️⃣ `immune_and_pd_model`

* This is the function defining your system:

```python
def immune_and_pd_model(y, t, params, van_func, lzd_func, immune_model):
```

* `y` is the current state vector: `[S, R, A_res, N]` (sensitive bacteria, resistant bacteria, reservoir, neutrophils).
* `t` is the current time.
* `params` contains all the biological and pharmacodynamic constants.
* `van_func` and `lzd_func` are functions giving the drug concentrations at any time `t`.
* `immune_model` is your separate immune response class, which calculates neutrophil growth and bacterial killing.

The function **returns a list of derivatives** `[dS/dt, dR/dt, dA_res/dt, dN/dt]`. `odeint` uses these derivatives to step forward in time.

---

### 3️⃣ `y0`

* Initial values for the ODEs:

```python
y0 = [1e3, 1e3, 1e1, immune_model.N0]
```

* Sensitive bacteria start at `1000`, resistant at `1000`, reservoir at `10`, neutrophils at `N0` (5000).

---

### 4️⃣ `t_eval`

* Array of times at which you want the solution.
* For example:

```python
t_eval = np.linspace(0, total_h, 500)
```

* This generates 500 equally spaced points from `t=0` to `t=200h`.

---

### 5️⃣ `args=(...)`

* Extra arguments for your derivative function.
* `odeint` only passes `y` and `t` by default, so you provide everything else (`params`, `van_func`, `lzd_func`, `immune_model`) here.

---

### 6️⃣ Output: `solution`

* This is a **2D NumPy array** with shape `(len(t_eval), len(y0))`.

* Each **row** corresponds to a time point in `t_eval`.

* Each **column** corresponds to one state variable:

  * Column 0 → Sensitive bacteria `S`
  * Column 1 → Resistant bacteria `R`
  * Column 2 → Reservoir `A_res`
  * Column 3 → Neutrophils `N`

* You can then plot these columns against `t_eval` to visualize dynamics over time.

---

In short, this single line:

1. Tells Python the system of ODEs.
2. Gives initial conditions and time points.
3. Solves the system numerically.
4. Returns a solution array for plotting or analysis.

---



Notes on  `ImmuneResponse` class. 

```python
class ImmuneResponse:
    def __init__(self, rho_N=1e-6, N_MAX=30000, delta_N=0.03, kill_N=5e-5):
        self.rho_N = rho_N
        self.N_MAX = N_MAX
        self.delta_N = delta_N
        self.kill_N = kill_N
```

---

### 1. `rho_N = 1e-6`

* This is the **proliferation rate of neutrophils**, per unit bacteria per unit time.
* It means neutrophils can expand in response to infection.
* Since it’s small (`1e-6`), neutrophil proliferation is very slow compared to bacterial growth rates.
* Biologically: neutrophils multiply **in proportion to bacterial load** — more bacteria = stronger stimulus for neutrophil growth.

---

### 2. `N_MAX = 30000`

* This is the **carrying capacity (maximum population)** of neutrophils in the system.
* It prevents neutrophils from growing without bound.
* Think of it like “bone marrow supply limits” or “physical maximum neutrophil concentration.”
* In the logistic growth term:

  $$
  \rho_N \cdot N \cdot B \cdot \Big(1 - \frac{N}{N_{MAX}}\Big)
  $$

  * When $N \ll N_{MAX}$, neutrophils can grow.
  * When $N \approx N_{MAX}$, growth → 0.

---

### 3. `delta_N = 0.03`

* This is the **decay rate (death/turnover)** of neutrophils.
* Without bacteria to stimulate growth, neutrophils decline at this rate.
* A value of `0.03` means about **3% of neutrophils die per unit time** (depending on time units in your model, e.g. per hour).

---

### 4. `kill_N = 5e-5`

* This is the **killing efficiency of neutrophils against bacteria**.
* Appears in the equations as terms like:

  $$
  - \text{kill}_N \cdot N \cdot S, \quad - \text{kill}_N \cdot N \cdot R
  $$
* It says each neutrophil can kill a certain fraction of bacteria per unit time.
* Larger `kill_N` → more effective immune clearance.

---

### 🔗 Putting it together

* Neutrophils grow in response to bacteria (`rho_N`) but are limited by a maximum (`N_MAX`).
* They also die off naturally (`delta_N`).
* Their presence reduces bacterial populations (`kill_N`).

So the class simply **stores the immune system parameters** that get used in the ODE system.

---

Would you like me to also **show the exact ODE for neutrophils with these parameters plugged in**, so you can see how each parameter directly shapes the curve?


Ah — thanks for clarifying. Let’s break down exactly what

```python
def sample_parameters(base_params, sigma=0.3):
    sampled_params = {}
    for k, v in base_params.items():
        if k not in ['B_max','k_res']:
            mu = np.log(v) - 0.5 * sigma**2
            sampled_params[k] = np.random.lognormal(mean=mu, sigma=sigma)
        else:
            sampled_params[k] = v
    return sampled_params
```

is doing:

---

### 1. Purpose

This function **samples a new set of parameters** from a probability distribution for use in Monte Carlo simulation.
It introduces randomness around the "base" values so that repeated simulations show variability.

---

### 2. Looping through parameters

* `base_params` is a dictionary of all model parameters (growth rates, kill rates, etc.).
* For each parameter `k` with value `v`, the function decides **how to sample it**.

---

### 3. Lognormal sampling (default case)

For most parameters:

```python
mu = np.log(v) - 0.5 * sigma**2
sampled_params[k] = np.random.lognormal(mean=mu, sigma=sigma)
```

* Parameters are assumed to follow a **log-normal distribution**.

* Why log-normal?

  * Many biological rates are positive and often vary multiplicatively (e.g., a rate might double or halve).
  * A log-normal ensures the sampled values are always positive.

* **Conversion details:**

  * If a variable $X$ is log-normally distributed:

    $$
    \ln(X) \sim \mathcal{N}(\mu, \sigma^2)
    $$
  * The mean of $X$ in the original scale is:

    $$
    \text{E}[X] = e^{\mu + 0.5\sigma^2}
    $$
  * To make the *expected value* equal to the original base value $v$, we set:

    $$
    \mu = \ln(v) - 0.5\sigma^2
    $$

✅ This ensures that the average sampled value is **centered around `v`**.

---

### 4. Special case (fixed parameters)

```python
if k not in ['B_max','k_res']:
    ...
else:
    sampled_params[k] = v
```

* Some parameters (`B_max`, `k_res`) are **not varied** across simulations.
* They are kept fixed at their base value.

This is probably because they represent fixed biological limits (like maximum bacterial load or transition constant) that you don’t want to treat as uncertain.

---

### 5. Output

The function returns a dictionary `sampled_params` that looks just like `base_params` but with randomness introduced for most parameters.

---

📌 **Summary in plain words:**
This function takes the baseline parameter set, and for each parameter except `B_max` and `k_res`, it replaces the value with a random draw from a log-normal distribution centered on the original value. This makes each simulation slightly different, capturing biological variability and uncertainty.

---

Do you want me to also **show visually** (e.g., histograms) how the distribution of a single parameter (say, growth rate) looks after this sampling? That would explain why the boxplots spread so wide.
